# TASK 1.1

In [178]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler, Normalizer
df = pd.read_csv('data.csv')
df = df.dropna()

df = df.drop(['education'], axis=1)
dataset =None



print("La distribución de la variable objetivo es:\n",df['TenYearCHD'].value_counts())
X = df.drop(['TenYearCHD'], axis=1).to_numpy()

La distribución de la variable objetivo es:
 0    3099
1     557
Name: TenYearCHD, dtype: int64


# TASK 1.2


In [179]:
from sklearn.decomposition import PCA
from scipy.stats import shapiro



variables_significativas =[]
pca = PCA(n_components=6)

X = pca.fit_transform(X)

for i in range(X.shape[1]):
    stat, p = shapiro(X[:,i])
    alpha = 0.05
    variables_significativas.append(df.columns[i])
    if p > alpha:
        print(f'La columna {df.columns[i]} parece provenir de una distribución normal')
    else:
        print(f'La columna {df.columns[i]} no parece provenir de una distribución normal')
        
print("\nLas varaibles significativas",variables_significativas)
variables_significativas.append('TenYearCHD')

dataset = df[variables_significativas]



La columna sex no parece provenir de una distribución normal
La columna age no parece provenir de una distribución normal
La columna currentSmoker no parece provenir de una distribución normal
La columna cigsPerDay no parece provenir de una distribución normal
La columna BPMeds no parece provenir de una distribución normal
La columna prevalentStroke no parece provenir de una distribución normal

Las varaibles significativas ['sex', 'age', 'currentSmoker', 'cigsPerDay', 'BPMeds', 'prevalentStroke']


# TASK 1.3

In [180]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
import warnings

warnings.filterwarnings("ignore")


y = dataset['TenYearCHD']
x = dataset.drop(['TenYearCHD'], axis=1)


sm = SMOTE(random_state=1234)
x,y = sm.fit_resample(x, y)
print("La nueva distribución de la variable objetivo es:\n",y.value_counts())


X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1234)


poly = PolynomialFeatures(degree=2)

X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.fit_transform(X_test)

logreg = LogisticRegression(max_iter=500, random_state=1234)

logreg.fit(X_train_poly, y_train)

y_pred = logreg.predict(X_test_poly)

accuracy = accuracy_score(y_test, y_pred)
print(f'\nPrecisión del modelo con un grado 2 es: {accuracy:.2f}')

La nueva distribución de la variable objetivo es:
 0    3099
1    3099
Name: TenYearCHD, dtype: int64

Precisión del modelo con un grado 2 es: 0.65


# TASK 1.4

In [181]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# Definir la cuadrícula de hiperparámetros
param_grid = {'poly__degree': [1, 2, 3, 4, 5]}

# Definir el modelo
model = LogisticRegression(max_iter=500,random_state=1234)

# Definir el pipeline
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),
    ('logreg', model)
])

# Realizar la búsqueda de hiperparámetros
grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)


# Mostrar los resultados
print(f'Mejor grado de polinomio: {grid_search.best_params_["poly__degree"]}')
print(f'Precisión de la validación cruzada con el mejor grado de polinomio: {grid_search.best_score_:.2f}')


Mejor grado de polinomio: 2
Precisión de la validación cruzada con el mejor grado de polinomio: 0.64


Realizando nuevamente el modelo pero utilizando los valores obtenidos en el paso anterior, se obtiene el siguiente resultado:

In [182]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
import warnings

warnings.filterwarnings("ignore")


poly = PolynomialFeatures(degree= grid_search.best_params_["poly__degree"])

X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.fit_transform(X_test)

logreg = LogisticRegression(max_iter=500, random_state=1234)

logreg.fit(X_train_poly, y_train)

y_pred = logreg.predict(X_test_poly)

accuracy = accuracy_score(y_test, y_pred)
print(f'\nPrecisión del modelo con un grado {grid_search.best_params_["poly__degree"]} es: {accuracy:.2f}')


Precisión del modelo con un grado 2 es: 0.65


# TASK 1.5

Utilizando PCA con 6 componentes se obtuvieron que las features más importantes son:
sex, age, currentSmoker, cigsPerDay, BPMeds, prevalentStroke. Si utilizamos más componentes la metrica de desempeño empeora. Education fue retirada desde el inicio ya que la educación no es un factor que influya en la probabilidad de tener un infarto de forma directa.

La primera vez que realice el modelo se me olvido verificar si el dataset estaba balanceado lo que me llevo a tener overfiting. Para dicha iteración el modelo obtuvo un accuracy de 0.89 ya que era muy bueno para predecir cuando el paciente no tuvo infarto.Como el dataset no estaba balanceado opte por utilizar SMOTE para generar datos y asi balancear el dataset. Una vez balanceado repeti el proceso y el modelo con grado polinomial 2 obtuvo un accuracy de 0.65. Luego mediante GridSearchCV se obtuvo que el mejor modelo es con grado polinomial 2 y con un accuracy de 0.65 igual en mi primer intento. Como conclusión se puede decir que el modelo no es bueno para predecir cuando un paciente va a tener un infarto, a pesar de que en el analisis inicial las variables parcian ser muy significativas respecto a la varaible obejtivo.